# The `swifttools.ukssdc.query.GRB` class

The `GRBQuery` class is a child class of the [`swifttools.ukssdc.query` class](../query.ipynb) extending its functionality to give some GRB-specific options. It makes use of the [`swifttools.ukssdc.data.GRB` module](../data/GRB.ipynb) to allow you to download GRB data products for objects found by querying.

This combination of features means that you can now very easily carry out tasks such as downloading all XRT light curves for GRBs with T90&lt;2 s - something many people have requested.

In this guide I am going to cover the GRB-specific query features and show you some examples of how to get data, but I am not going into all the details of the generic query syntax, or the product access functions. For those I refer you to the [`query`](../query.ipynb) and [`data.GRB`](../data/GRB.ipynb) documentation.

First we will import the module:

In [ ]:
import swifttools.ukssdc.query as uq

## Page contents

* [The `GRBQuery` class](#grbquery)
* [The catalogues](#cats)
* [Combining catalogues](#aux)
* [GRB Products](#prods)
  * [Light curves](#curves)
  * [Spectra](#spectra)
  * [Burst analyser](#ban)
  * [Positions](#positions)
  * [Obs data](#data)

----

<a id='grbquery'></a>
## The `GRBQuery` class

For querying the GRB catalogues we use the `GRBQuery` class. We can create an object of this just like [we did for `ObsQuery`](../query.ipynb#obsquery):

In [ ]:
q = uq.GRBQuery(silent=False)

I've again set `silent=False` as for interactive, and especially pedagogical use, this is helpful.

Whereas for `ObsQuery` we had a choice of which table in the catalogue we wanted to query, for `GRBQuery` the situation is slightly different, we have to first choose which GRB catalogue we want to query. At the present time, there is only one table in each catalogue, so this effectively replaces the table controls we saw for `ObsQuery`. Before we go any further, I should introduce these catalogues.


<a id='cats'></a>
## The catalogues

There is only one GRB catalogue produced by the UKSSDC, and this is limited to GRB data. However, as there has been (much!) demand for a way to access XRT products for samples derived from other information, I've provided tools to use two other catalogues provided by [the SDC](https://swift.gsfc.nasa.gov) as well. So, the `GRBQuery` module lets use select the following catalogues:

* [The live XRT GRB catalogue](https://www.swift.ac.uk/xrt_live_cat) "UK_XRT"
* [The SDC Data Table](https://swift.gsfc.nasa.gov/archive/grb_table.html/) "SDC_GRB"
* [The Swift/BAT GRB catalog](https://swift.gsfc.nasa.gov/results/batgrbcat/) "SDC_BAT"

The labels at the end of each row above are the names by which these catalogues are accessed, as you'll see in a moment.

Because the SDC tables are provided externally (i.e. by the SDC), in order to fit them nicely into my API back end, I have actually set up CRON jobs that run hourly to download the latest versions from the above sources and ingest them into my own database system. This means that there is always the possibility that things are slightly out of sync, but never by more than an hour. I also created the metadata myself, so any errors therein are entirely my own fault.

There are also a couple of warnings and caveats related to these tables.

The Swift Data Table is curated by JD Myers at GSFC, and is essentially manually compiled from information contained in GCN circulars. This has presented the occasional challenge ingesting everything into a form that can be queried by this API. For example, how will a T90 value of "~2" fare with a ">" operator? To resolve this, I strip out the non-numeric characters from columns like this, and set some warning columns [which we will look at presently](#dtcaveat).

The BAT GRB catalogue is a machine-readable catalogue without these issues and so is just ingested directly, but again there a couple of warnings. First and most importantly, at the time of writing the catalogue only includes GRBs up to the end of 2023. Secondly, when I ingested that table I found a couple of GRBs with duplicate rows; only the first row is ingested ('name' is required as a unique column for combining catalogues).

Right, with those preliminaries out of the way, let's get to business.

Managing which catalogue we are querying is directly analogous to managing tables, except that we replace "table" with "cat". So (in case you didn't to it above) let's create a query:

In [ ]:
q = uq.GRBQuery(silent=False)

And then find out which catalogue was selected by default.

In [ ]:
q.cat

I listed all of the catalogues above, but you can get their labels directly from the class too:

In [ ]:
q.cats

We can change catalogue in two ways, either changing the cat variable, or supplying it to the constructor. For example:

In [ ]:
q = uq.GRBQuery(cat="BAT_GRB", silent=False)
q.cat = "SDC_GRB"

So in the above I created a query using the BAT GRB catalogue, and then decided I actually wanted the SDC data table catalogue instead.

You may be wondering why this is a catalogue and not a "table", whereas in the `ObsQuery` case we used tables. The answer is that the `query` module actually has both, and the heirarchy is `catalogue` -> `tables`. In the `ObsQuery` module we have a single catalogue that has multiple tables; in this case we have multiple catalogues that each have (at present) a single table. [The `SXPSQuery` class](SXPS.ipynb) contains multiple catalogues with multiple tables each.

Of course, we can jump right in with a simple query now, but I'm guessing you're not often going to want to select GRB data by a cone search and so the 'advanced' method will be more useful, and for that you will probably want to check out the metadata.

In [ ]:
q.metadata

Here I've shown the metadata for the SDC Data Table, and slightly annoyingly, Jupyter compresses the output. You can explore it yourself of course, or we can just extract the 'ColName' data, i.e. what columns does the SDC Data Table supply?

In [ ]:
list(q.metadata["ColName"])

<a id='dtcaveat'></a>
### SDC data table caveats

For the "SDC_GRB" table used above there are some caveats which I've alluded to above - some fields in the original table cannot be directly ingested into my database. The columns ending '\_warn' and '\_orig' which you can see above tell us when that happened. Any time a field cannot be ingested 'as is' (for example a numerical field that contained a non-numerical character), then the '\_warn' column for that row is set to `True` and the '\_orig' field is populated - this is a text field containing the original string. So, for example, if there was a problem with "BAT_T90" then the "BAT_T90" column will be replaced with my code's "best guess" as to the correct numerical value, "BAT_T90_orig" will be set to the original value, and "BAT_T90_warn" will be 1.

The last 4 columns listed in the metadata are summaries, they tell you if any warnings related to each instrument, or any warnings at all, were set. So one may, for example, want to filter out such rows, either in the query or after getting the results. Let's take a moment to explore this just a bit more.

First, we'll get all the data for a query and then filter it:

In [ ]:
q.addFilter(("BAT_T90", "<", 2))
q.addCol("*")
q.submit()
q.results

I could now define a subset of only those with no BAT_T90_warning value set, so I only have the cases that definitely match my query. Subsets were [discussed in the `ObsQuery` documentation](../query.ipynb#subsets).

In [ ]:
subset = ~q.results["BAT_T90_warn"].astype(bool)

(Quick Python note: to build the subset on a boolean column we don't need a comparison, so `q.results['some_col']` returns the indices where `some_col` is `True`, the `~` above returns instead where it is `False`. The 'boolean' columns in the database are actually integers with values 1 or 0 (blame MariaDB) hence the `astype()` call).

I could pass this subset to any of the product functions that we will come to later, or just take a look at it:

In [ ]:
q.results.loc[subset]

This only has one fewer row than the original query. It may be more informative to look at that row, so lets find the row which did have the warning set:

In [ ]:
subset = q.results["BAT_T90_warn"].astype(bool)
myFrame = q.results.loc[subset]
myRow = myFrame.iloc[0]
print(myRow["Name"])
print(myRow["BAT_T90"])
print(myRow["BAT_T90_orig"])

Here you can see the problem: GRB 161004A has a T90 value in the SDC_GRB table of "~1.3 to ~3". Since I want T90 to be a number, my ingestion code took "1.3" but set the warning flag. 

How you choose to handle such cases is entirely up to you, and depends on your specific needs; all I'm trying to do here is to show you the tools I have provided to help you in this.

<a id='aux'></a>
## Combining catalogues

While all of these catalogues are, in themselves, great, I think we're often going to want to select GRBs using data from multiple catalogues. For example, maybe you want to get all short GRBs with at least one break in their XRT light curve, or something like that. In this case, we're going to have to combine the catalogues. And for this purpose the `GRBQuery` class includes the concept of auxilliary catalogues. The premise here is very simple: you define a `GRBQuery` and select the catalogue to query on, I will call this the 'primary catalogue'. Then you can add a second catalogue, which we call the 'auxilliary catalogue'. Then you define filters and submit, and the only results returned will be those which met the criteria in both catalogues.

As always an example teaches better than my blethering, so let's do exactly what I just said, get all GRBs with T90<2 and at least one break in their XRT light curve fit.

I'm going to use the BAT GRB catalogue for T90 (even though it stops in 2020) for this demo and I'll make that the primary catalogue, although which is primary and which auxilliary makes very little difference.

First up, let's create a query object for the primary catalogue. As ever for the demos, I'll turn off silent mode.

In [ ]:
q = uq.GRBQuery(cat="BAT_GRB", silent=False)

Now I will add the XRT catalogue as be an auxilliary:

In [ ]:
q.setAuxCat("UK_XRT")

`setAuxCat()` requires a catalogue  name, and optionally `silent` and `verbose`; if these latter are not specified, they are set to the values of the primary catalogue.

The auxilliary catalogue is literally just another `GRBQuery` object, and we can access it via `q.auxCat` so anything we can do to the primary catalogue we can also do to the auxilliary catalogue. This is handy because we need to add filters to both.

**Important note** filters must be added to the correct catalogue; I have not made a mechanism to infer from the column name which catalogue you meant, because it is possible for both catalogues to share column names. So it is your reponsibility to get the right filters in the right place. 

So, let's do that. FIrst we want a T90 filter, which applies to our primary catalogue:

In [ ]:
q.addFilter(("T90", "<", 2))

and we want a filter on the number of light curve breaks for the auxilliary catalogue:

In [ ]:
q.auxCat.addFilter(("NumLCBreaks", ">=", 1))

If you're wondering what the filter syntax should be, or how I knew the names of the columns to filter on, you should read the [top-level `query` documentation](../query.ipynb) which explains both of these things.

Now we can submit the query.

In [ ]:
q.submit()

Because we had `silent=False` you can see that there were two queries done, and they received different number of rows, however:

In [ ]:
print(len(q.results))
print(len(q.auxCat.results))

You can also see that the results - of both catalogues - have been filtered to contain only matches. By default, the results are kept separate:

In [ ]:
q.results

In [ ]:
q.auxCat.results

We can merge these into a single table. This will change `q.results` (but not `q.auxCat.results`).

In [ ]:
q.mergeResults()

q.results

We could actually have done this at submit time if we wanted, let's redo the query and demonstrate that. I can't run `submit` again without unlocking or resetting the query. I'll reset it and show you a couple of things while I'm here:

In [ ]:
q.reset(keepAux=True, keepFilters=True)

This reset the query, but kept all my filters and the auxilliary catalogue (although that too was reset). If I'd also defined which columns to retrieve I could have kept them using the `keepCols` argument.

For our new query, let's request all columns in both catalogues:

In [ ]:
q.addCol("*")
q.auxCat.addCol("*")

In [ ]:
q.submit(merge=True)

Note that this time I gave `merge=True`, so we should have merged the results already:

In [ ]:
q.results

And indeed we have. I know Jupyter is truncating the output, but I also know that `Trig_ID` is a BAT_GRB column and `Onboard_Decl_apy` is from the UK_XRT catalogue, so I can see that it worked. Oh and by the way, do note that as for ObsQuery, for all the RA/Dec columns we have `_s` (=sexagesimal) and `_apy` (=astropy) columns created for us.

That's really the bulk of auxilliary catalogues and queries covered. I didn't cover cone searches because they are so simple and covered in the [`ObsQuery` tutorial](../query.ipynb); all I will add is that if you run `q.addConeSearch()` to a query with an auxCat then the cone search will be automatically applied to the aux cat as well *provided you've already added the aux cat*. Essentially I always advise that the very first things you do are create your query and add an auilliary catalogue if you need to, and then add filters etc.

Lastly, since the `auxCat` is itself just a `GRBQuery` object, it too can have an `auxCat`. So you can query by combining all three catalogues if you want (i.e. `q.auxCat.setAuxCat()`), although I haven't tested this. One note of warning for this: when you add an auxilliary catalogue you will be prevented from choosing the same catalogue as the primary one. If you try to add a third layer you can actually use the primary catalogue again. i.e. you could have XRT_UK -> BAT_GRB -> XRT_UK. If you do this, you deserve whatever happens.

<a id='prods'></a>
## GRB Products

Having identified a sample of GRBs, we may want to actually get at some of the data - recall my example above where I've been asked for the ability to *download all XRT light curves* for GRB with T90&lt;2 s. 

This is easy to do, because the `GRBQuery` class provides wrappers to all of the [`swifttools.ukssdc.data.GRB` functions](../data/GRB.ipynb). As [already explained for `ObsQuery`](../query.ipynb#prods), the syntactic difference is just that we don't provide the list of objects to retrieve, that is automatically taken from `q.results`, but we can provide [a subset](../query.ipynb#subsets) of rows.

I also remind you that I am *not* going to spend ages detailing all the different options available for downloading products and what they do. I did it in the [`data.GRB` documentation](../data/GRB.ipynb) so you can refer to that for details.

The other thing to remind you is that by default all of the functions to get data (starting `get`), when called via the `query` module, neither save data to disk nor return it, but save it in a variable inside your `GRBQuery` object. You can change this behaviour with the `saveData` and `returnData` arguments, but even then, the data will still be stored in class variables. I will introduce those variables to you in a moment, but first let me tell you something about them. They are always  `dict`s with one key per object in your query results, even if there was only one object found. You can decide whether the key is the GRB name or targetID by specifying **one** of `byName=True` or `byID=True` (if you specify neither, name is assumed). Of course, your results must include the specified column.

So, the basic syntax of every product retrieval function is the same:

`q.get<something>(byID, byName, subset, returnData, saveData, **kwargs)`

where `**kwargs` are any arguments you want to pass to the underlying function in [`data.GRB`](../data/GRB.ipynb).

So, let's run some demos. We'll stick with the query above, but I'll repeat the cells here in case you haven't run them or have been doing your own editing:

In [ ]:
q = uq.GRBQuery(cat="BAT_GRB", silent=False)
q.setAuxCat("UK_XRT")
q.addFilter(("T90", "<", 2))
q.auxCat.addFilter(("NumLCBreaks", ">=", 1))
q.submit(merge=True)
print(f"\n\nI have {len(q.results)} rows in the merged table")

<a id='curves'></a>
### Light curves

Let's open up just by getting light curves for everything. And I'll index them by targetID instead of name because why not?

In [ ]:
q.getLightCurves(incbad=True, byID=True)

I added `incbad` just to prove that `**kwargs` works.

As you should have antipicated, nothing was returned by the function and nothing written to disk. Our light curve data is in our `q` object, in a variable called, oddly enough, `lightCurves`. As I've explained a moment ago, this should be a `dict` with one entry for each of the 16 GRBs matching our query, and each of those should be a [light curve `dict`](https://www.swift.ac.uk/API/ukssdc/structures.md#the-light-curve-dict).

In [ ]:
list(q.lightCurves.keys())

That looks about right, except the strange entry of '0', but actually (I checked this) one of the GRBs in the BAT table did have a triggerID of 0. I don't know why, but there you are. There is no light curve for this object:

In [ ]:
q.lightCurves[0]

so we will chalk this one up as a mystery and move on.

The other entries are standard light curve `dict`s as expected, e.g.:

In [ ]:
list(q.lightCurves[821103].keys())

If we call `getLightCurves` again it will combine the results with this `dict`, so imagine I realise I wanted the non-"incbad" data:

In [ ]:
q.getLightCurves(incbad=False, byID=True)
list(q.lightCurves[821103].keys())

and you can see that the light curve `dict` has been updated.

We can also completely forget the light curves with `clearLightCurves()`, so let's do that and just check what happens if we supply no arguments to `getLightCurves()`

In [ ]:
q.clearLightCurves()
q.getLightCurves()

Given that I didn't supply `byID` or `byName` what has happened? Let's have a look:

In [ ]:
list(q.lightCurves.keys())

For me, I see GRB names in the list above, **I can't guarantee that you will see the same** there is no default set so what you see depends on how your Python executable traverses the internal data. For the following examples I'm assuming the results are indexed by name, so if yours are not, then run:

In [ ]:
q.clearLightCurves()
q.getLightCurves(byName=True)

(By the way, all those "Resolved" lines, which are suppressed if `silent=True` are just because you are getting the GRBs by name, but on the UKSSDC they are indexed by targetID, so some look ups are being done.)

#### Saving light curves

Although the default behaviour of the `query` module is to save data to an internal variable, we can still save it to disk. One way would be to say `saveData=True`, and then pass all the arguments like `destDir` etc. This was covered in detail in the [the `data.GRB` notebook](../data/GRB.ipynb#lightcurves), so you can read that if you want to know how.

The query module also lets us save the light curves after downloading. Again, this is very similar to the `data` module, except that we have an extra argument: `whichGRBs`. This optional argument takes a list/tuple of the light curves in our `q.lightCurves` variable, and lets us save only some of the downloaded curves:

In [ ]:
q.saveLightCurves(whichGRBs=["GRB 051221A", "GRB 100117A"], destDir="/tmp/APIDemo_GRB_LC", header=True, subDirs=True)

#### Plotting light curves

You can also plot light curves, using the [module-level `plotLightCurve()` function](https://www.swift.ac.uk/API/ukssdc/commonFunc.md#plotlightcurve), but because I'm really nice, and you really want to buy me a drink, I've added a `plotLightCurves()` function into this module to wrap around it - although it still only plots one LC at a time. 


If we don't try to specify the datasets to plot, we may end up in a mess (or at least, with a really messy plot), so let's pick a GRB and check what we have:

In [ ]:
q.lightCurves["GRB 060313"]["Datasets"]

OK, now let's plot this:

In [ ]:
q.plotLightCurves("GRB 060313", whichCurves=("WT_incbad", "PC_incbad", "PCUL_incbad"), xlog=True, ylog=True)

As noted in the [`plotLightCurve()` documentation](https://www.swift.ac.uk/API/ukssdc/commonFunc.md#plotlightcurve), it returns the pyplot `fig` and `ax` objects, and can receive them as well, which means we can add extra light curves to the same plot. So, let's repeat the above (capturing the return) and then we'll add a second light curve:


In [ ]:
f, a = q.plotLightCurves("GRB 060313", whichCurves=("WT_incbad", "PC_incbad", "PCUL_incbad"), xlog=True, ylog=True)

In [ ]:
f, a = q.plotLightCurves(
    "GRB 160501A",
    whichCurves=("WT_incbad", "PC_incbad", "PCUL_incbad"),
    xlog=True,
    ylog=True,
    fig=f,
    ax=a,
    cols={"WT": "cyan", "PC": "magenta"},
)
f

For reasons I don't entirely follow, if the above cell doesn't end with the `f` (i.e. just the `matplotlib.figure` object, Jupyter doesn't plot it. 

Notice above I made use of the `cols` argument to make the second GRB use difference colours to the first.

The purpose of `plotLightCurve()` and its wrappers is, however, not to give you the perfect plotting function to do anything you want; it's there to give an easy and quick way of plotting light curves, so you can have a look at what's going on. If you want to do anything beyond this then you'll need to write your own functions.

<a id='spectra'></a>
### Spectra

This will shock you I'm sure, but to get spectra we replace the word 'lightCurves' in the above example with 'spectra'. I know, outrageous, right? Let's throw in a subset just to justify this being a separate example, and I'll also remind you how `saveData` works because that is how nice I am.

In [ ]:
q.getSpectra(
    subset=q.results["Err90"] < 1.9,
    saveData=True,
    destDir="/tmp/APIDemo_GRB_Spec2",
    extract=False,
    removeTar=False,
    saveImages=True,
)

This function has done a few things. Firstly, it only got spectra for rows where the "Err90" column was less than 1.9&dagger;.

Secondly, it saved the spectral data for those objects, in the form of `tar` files and images, to '/tmp/APIDemo_GRB_Spec2', but it neither extracted the `tar` files nor deleted them. And lastly, it saved the spectral data to the internal variable `q.spectra` (this last point was not requested explicitly, it *always* happens). 

(&dagger; To find out what Err90 is, you'd have to read the metadata, or in this case, the auxCat metadata. To save you the bother: it's the XRT 90% confidence radial position error, in arcsec.)

Let's check out that variable:

In [ ]:
list(q.spectra.keys())

Each of these will contain a [spectrum `dict`](https://www.swift.ac.uk/API/ukssdc/structures.md#the-spectrum-dict) and we explored that in the [`data.GRB` documentation](../data/GRB.ipynb#spectra) so I'm not repeating it here.

As with lightcurves, we can save the data after downloading as well, specifying which GRBs to save if we wish:

In [ ]:
q.saveSpectra(
    destDir="/tmp/APIDemo_GRB_Spec3",
    whichGRBs=(
        "GRB 051221A",
        "GRB 060218",
        "GRB 060313",
        "GRB 061201",
    ),
    extract=True,
    removeTar=True,
)

<a id='ban'></a>
### Burst analyser

Honestly, nothing about this should be difficult or surprising. Again, all the arguments available to [`data.GRB.getBurstAnalyser()`](../data/GRB.ipynb#ban) exist and the wrapper in this class works just like those above:

In [ ]:
q.getBurstAnalyser(
    subset=q.results["Err90"] < 1.5,
    downloadTar=True,
    extract=False,
    removeTar=False,
    destDir="/tmp/APIDemo_GRB_burstAn",
)

This time I got the results only for things with Err90 below 1.5 (arcsec), and again, saved them to disk. And as you should be aware, the data, in the form of [a burst analyser `dict`](https://www.swift.ac.uk/API/ukssdc/structures.md#the-burst-analyser-dict), were also saved into a class variable whose name I'm sure you can hazard a guess at:

In [ ]:
q.burstAnalyser.keys()

And again, we can also save data having downloaded it:

In [ ]:
q.saveBurstAnalyser(
    destDir="/tmp/APIDemo_GRB_burstAn2",
    whichGRBs=["GRB 060218", "GRB 200324A"],
    header=True,
    subDirs=True,
    usePropagatedErrors=True,
    instruments=[
        "XRT",
    ],
)

<a id='pos'></a>
### Positions

GRB positions differ only from the above examples by the fact that there is no `saveData` option. If I'm honest, I don't know why you may want to get positions for GRBs returned by a query, because they are already in the 'UK_XRT' catalogue, so you can just return those columns, but hey, I'm not here to judge (much):

In [ ]:
q.getPositions(byName=True, subset=q.results["Image_position_err"] > 1.8)

This time I made a subset based on the BAT image position error (in arcmin) just for fun, and I got 5 results. These were saved in (can you guess?) `q.positions`

In [ ]:
q.positions.keys()

In [ ]:
q.positions["GRB 060218"]

Before moving on let me use positions, as they are small, to show one other thing I mentioned but haven't demostrated: all of this `get` functions can take `returnData=True`, to return the data as well as storing it internally:

In [ ]:
pointlessVar = q.getPositions(byName=True, subset=q.results["Image_position_err"] > 1.8, returnData=True)
pointlessVar.keys()

<a id='data'></a>
### Obs Data

And finally, you may want to download all of the obsData for your result. In this demo I will literally only get one GRB, to speed things up a bit. This is the one `get` function that sets no interval variable, it only saves data to disk:

In [ ]:
q.getObsData(
    destDir="/tmp/APIDemo_GRBdata",
    subset=q.results["GRBname"] == "GRB 200324A",
    instruments=[
        "XRT",
    ],
)

## And last

If you're wondering where the options to rebin light curves or time-slice spectra are then I'm afraid you're going to be disappointed. Making it easy for you to (deliberately or accidentally) request time-sliced spectra for 1,500 GRBs is not something I want to do -- we do, after all, have a finite compute load. However, it should be pretty easy for you to write a code to loop over your results and submit such jobs, one or two at a time. I would do it now as a demo, but I'm not *that* nice :)